https://github.com/huggingface/transformers/blob/0ae96ff8a7e2d371242452d81bee85da8df202f5/examples/text-generation/run_generation.py

# get a bunch of user comments

In [1]:
import praw

In [2]:
reddit = praw.Reddit()

In [3]:
user_name = 'suncoasthost'

In [4]:
user = reddit.redditor(user_name)

In [5]:
def get_context(comment, reddit):
    istoplevel = comment.parent_id == comment.link_id
    submission = reddit.submission(id=comment.link_id)
    parent_comment = reddit.comment(id=comment.parent_id.replace('t1_', '')) if not istoplevel else None
    return parent_comment, submission

In [6]:
comment_context = [(comment, get_context(comment, reddit)) for comment in user.comments.new()]

In [7]:
comment_context[0]

(Comment(id='fw9xf8b'), (Comment(id='fw9win2'), Submission(id='t3_hgtxpz')))